# VAERS - COVID19
The idea of the current notebook is to assess the veracity of the data presented in https://www.openvaers.com/covid-data    
OpenVAERS website is not affiliated to neither https://vaers.hhs.gov nor https://wonder.cdc.gov, where VAERS data can be downloaded and explored, respectively.

## Filesystem cleanup

In [0]:
%sh mkdir -p /tmp/datasets/

In [0]:
%sh rm -rf /tmp/datasets/*

## Dataset preparation

**Get dataset**  
Manually download the 2020, 2021 and NonDomestic zip files from https://vaers.hhs.gov/data/datasets.html (the website has a captcha system)  
Inside it can also be found the **VAERS Data User Guide**, in case you want to know more about the datasets that are going to be used in this notebook.  

Upload the files to `dbfs:/FileStore/` so they can later be accessed from Databricks cluster.

In [0]:
%fs ls /FileStore/

path,name,size
dbfs:/FileStore/2020VAERSData.zip,2020VAERSData.zip,11393690
dbfs:/FileStore/2021VAERSData.zip,2021VAERSData.zip,89300030
dbfs:/FileStore/NonDomesticVAERSData.zip,NonDomesticVAERSData.zip,56636236
dbfs:/FileStore/import-stage/,import-stage/,0
dbfs:/FileStore/plots/,plots/,0
dbfs:/FileStore/tables/,tables/,0


In [0]:
dbutils.fs.cp("dbfs:/FileStore/2020VAERSData.zip", "file:/tmp/datasets/2020VAERSData.zip")
dbutils.fs.cp("dbfs:/FileStore/2021VAERSData.zip", "file:/tmp/datasets/2021VAERSData.zip")
dbutils.fs.cp("dbfs:/FileStore/NonDomesticVAERSData.zip", "file:/tmp/datasets/NonDomesticVAERSData.zip")

Out[4]: True

In [0]:
%sh 
unzip /tmp/datasets/2020VAERSData.zip -d /tmp/datasets/
unzip /tmp/datasets/2021VAERSData.zip -d /tmp/datasets/
unzip /tmp/datasets/NonDomesticVAERSData.zip -d /tmp/datasets/

Archive: /tmp/datasets/2020VAERSData.zip
 inflating: /tmp/datasets/2020VAERSDATA.csv 
 inflating: /tmp/datasets/2020VAERSSYMPTOMS.csv 
 inflating: /tmp/datasets/2020VAERSVAX.csv 
Archive: /tmp/datasets/2021VAERSData.zip
 inflating: /tmp/datasets/2021VAERSDATA.csv 
 inflating: /tmp/datasets/2021VAERSSYMPTOMS.csv 
 inflating: /tmp/datasets/2021VAERSVAX.csv 
Archive: /tmp/datasets/NonDomesticVAERSData.zip
 inflating: /tmp/datasets/NonDomesticVAERSDATA.csv 
 inflating: /tmp/datasets/NonDomesticVAERSSYMPTOMS.csv 
 inflating: /tmp/datasets/NonDomesticVAERSVAX.csv

In [0]:
%sh ls -lhR /tmp/datasets

/tmp/datasets:
total 795M
-rw-r--r-- 1 root root 41M Jul 4 17:05 2020VAERSDATA.csv
-rw-r--r-- 1 root root 11M Jul 13 14:10 2020VAERSData.zip
-rw-r--r-- 1 root root 4.5M Jul 4 17:05 2020VAERSSYMPTOMS.csv
-rw-r--r-- 1 root root 4.4M Jul 4 17:05 2020VAERSVAX.csv
-rw-r--r-- 1 root root 267M Jul 4 17:06 2021VAERSDATA.csv
-rw-r--r-- 1 root root 86M Jul 13 14:10 2021VAERSData.zip
-rw-r--r-- 1 root root 41M Jul 4 17:06 2021VAERSSYMPTOMS.csv
-rw-r--r-- 1 root root 30M Jul 4 17:06 2021VAERSVAX.csv
-rw-r--r-- 1 root root 230M Jul 4 17:07 NonDomesticVAERSDATA.csv
-rw-r--r-- 1 root root 55M Jul 13 14:10 NonDomesticVAERSData.zip
-rw-r--r-- 1 root root 18M Jul 4 17:07 NonDomesticVAERSSYMPTOMS.csv
-rw-r--r-- 1 root root 12M Jul 4 17:07 NonDomesticVAERSVAX.csv

In [0]:
%sh wc -l /tmp/datasets/*.csv

49413 /tmp/datasets/2020VAERSDATA.csv
 60672 /tmp/datasets/2020VAERSSYMPTOMS.csv
 59237 /tmp/datasets/2020VAERSVAX.csv
 406002 /tmp/datasets/2021VAERSDATA.csv
 542932 /tmp/datasets/2021VAERSSYMPTOMS.csv
 420982 /tmp/datasets/2021VAERSVAX.csv
 124523 /tmp/datasets/NonDomesticVAERSDATA.csv
 199100 /tmp/datasets/NonDomesticVAERSSYMPTOMS.csv
 169917 /tmp/datasets/NonDomesticVAERSVAX.csv
 2032778 total

In [0]:
%sh head -n2 /tmp/datasets/2021*.csv

==> /tmp/datasets/2021VAERSDATA.csv <==
VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,LAB_DATA,V_ADMINBY,V_FUNDBY,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0916600,01/01/2021,TX,33.0,33,,F,,Right side of epiglottis swelled up and hinder swallowing pictures taken Benadryl Tylenol taken,,,,,,,,,Y,12/28/2020,12/30/2020,2,None,PVT,,None,None,None,,,2,01/01/2021,,Y,,Pcn and bee venom

==> /tmp/datasets/2021VAERSSYMPTOMS.csv <==
VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0916600,Dysphagia,23.10,Epiglottitis,23.10,,,,,,

==> /tmp/datasets/2021VAERSVAX.csv <==
VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0916600,COVID19,MODERNA,037K20A,1,IM,LA,COVID19 (COVID19 (MODERNA))

**DataFrames and SQL views creation**

In [0]:
vaers_data = spark.read.csv("file:/tmp/datasets/*VAERSDATA.csv",
                     header='true',
                     sep=",",
                     escape='"',
                     dateFormat='MM-dd-yyyy',
                     timestampFormat='MM-dd-yyyy HH:mm:ss',
                     inferSchema='true')

In [0]:
vaers_data.createOrReplaceTempView("vaers_data")

In [0]:
vaers_symptoms = spark.read.csv("file:/tmp/datasets/*VAERSSYMPTOMS.csv",
                     header='true',
                     sep=",",
                     escape='"',
                     dateFormat='MM-dd-yyyy',
                     timestampFormat='MM-dd-yyyy HH:mm:ss',
                     inferSchema='true')

In [0]:
vaers_symptoms.createOrReplaceTempView("vaers_symptoms")

In [0]:
vaers_vax = spark.read.csv("file:/tmp/datasets/*VAERSVAX.csv",
                     header='true',
                     sep=",",
                     escape='"',
                     dateFormat='MM-dd-yyyy',
                     timestampFormat='MM-dd-yyyy HH:mm:ss',
                     inferSchema='true') \
                     .where("vax_type = 'COVID19'") \
                     .select("VAERS_ID", "VAX_MANU", "VAX_NAME")

In [0]:
# 451398 instead of 650133 (59236 + 420981 + 169916) from original files, which means that 198735 entries were not COVID19 vaccines.
vaers_vax.count()

Out[14]: 451398

In [0]:
# We will use DISTINCT to remove some strange duplicates, like the one below, which has the highest number of duplicates:
display(vaers_vax.filter("vaers_id = 1215401"))

VAERS_ID,VAX_MANU,VAX_NAME
1215401,JANSSEN,COVID19 (COVID19 (JANSSEN))
1215401,JANSSEN,COVID19 (COVID19 (JANSSEN))
1215401,JANSSEN,COVID19 (COVID19 (JANSSEN))
1215401,JANSSEN,COVID19 (COVID19 (JANSSEN))
1215401,JANSSEN,COVID19 (COVID19 (JANSSEN))
1215401,JANSSEN,COVID19 (COVID19 (JANSSEN))
1215401,JANSSEN,COVID19 (COVID19 (JANSSEN))
1215401,JANSSEN,COVID19 (COVID19 (JANSSEN))
1215401,JANSSEN,COVID19 (COVID19 (JANSSEN))
1215401,JANSSEN,COVID19 (COVID19 (JANSSEN))


In [0]:
# And now from 451398 to 438828, which means that 12570 entries were duplicates.
vaers_vax = vaers_vax.distinct()
vaers_vax.count()

Out[16]: 438828

In [0]:
vaers_vax.createOrReplaceTempView("vaers_vax")

Now an usual task would be to do some data wrangling to avoid `null` values, but since we don't know whether the user forgot to indicate some parameter or assumed that `null` already implied a negative value, we can't say for sure.  
Therefore, we will use `null` values as they were originally designed in databases, that is, as something unknown or undetermined.

In [0]:
%sql CREATE OR REPLACE TEMPORARY VIEW V_VAERS_FULL_DATA AS
SELECT 
  v.vax_manu, 
  d.vaers_id, d.recvdate, d.vax_date, d.age_yrs, d.sex, d.hospdays, d.disable, d.recovd, d.died, d.datedied, d.symptom_text
FROM vaers_data d
  JOIN vaers_vax v on d.vaers_id = v.vaers_id

In [0]:
%sql --Same number as in VAERSVAX dataset, which means we didn't add up extra rows from the join with VAERSDATA dataset.
select count(*) from V_VAERS_FULL_DATA

count(1)
438828


## Analysis 1 - Reports per vaccine

In [0]:
%sql select distinct(vax_manu) from vaers_vax

vax_manu
MODERNA
JANSSEN
UNKNOWN MANUFACTURER
PFIZER\BIONTECH


In [0]:
%sql select vax_manu, count(*) as reports
from vaers_vax
group by vax_manu
order by reports desc

vax_manu,reports
PFIZER\BIONTECH,208870
MODERNA,186532
JANSSEN,42359
UNKNOWN MANUFACTURER,1067


## Analysis 2 - Most common symptoms

In [0]:
%sql CREATE OR REPLACE TEMPORARY VIEW V_ALL_SYMPTOMS AS
select v.vax_manu, v.vaers_id,
       s.symptom1, s.symptom2, s.symptom3, s.symptom4, s.symptom5
from vaers_vax v
join vaers_symptoms s
  on v.vaers_id = s.vaers_id

It could happen that one report includes more than 5 symptoms, and that means that multiple rows could be included in Symptoms table for the same report.

In [0]:
%sql select * from V_ALL_SYMPTOMS
where vaers_id = 916691

vax_manu,vaers_id,symptom1,symptom2,symptom3,symptom4,symptom5
MODERNA,916691,Chills,Dizziness,Headache,Injection site pain,Nausea
MODERNA,916691,Productive cough,null,null,null,null


In order to do a proper analysis to all the symptoms, it would be easier if we had all of them in a single column, and for that we can use `stack` function:

In [0]:
%sql CREATE OR REPLACE TEMPORARY VIEW V_ALL_SYMPTOMS_STACK AS
select * from (
  SELECT vax_manu, vaers_id,
    stack(5, symptom1, symptom2, symptom3, symptom4, symptom5) as symptom
  FROM V_ALL_SYMPTOMS
  )
where symptom is not null

In [0]:
%sql select * from V_ALL_SYMPTOMS_STACK 
where vaers_id = 916691

vax_manu,vaers_id,symptom
MODERNA,916691,Chills
MODERNA,916691,Dizziness
MODERNA,916691,Headache
MODERNA,916691,Injection site pain
MODERNA,916691,Nausea
MODERNA,916691,Productive cough


Let's see now the most common symptoms for all 3 manufacturers separatedly, and using different output styles.

In [0]:
%sql 
select vax_manu, symptom, count(*) as reports 
from V_ALL_SYMPTOMS_STACK
where vax_manu = 'MODERNA'
group by vax_manu, symptom
order by reports desc
limit 50

vax_manu,symptom,reports
MODERNA,Headache,35673
MODERNA,Pyrexia,31665
MODERNA,Fatigue,29631
MODERNA,Chills,29486
MODERNA,Pain,28164
MODERNA,Nausea,21124
MODERNA,Injection site erythema,19842
MODERNA,Injection site pain,19234
MODERNA,Pain in extremity,19056
MODERNA,Dizziness,18745


In [0]:
%sql 
select vax_manu, symptom, count(*) as reports 
from V_ALL_SYMPTOMS_STACK
where vax_manu = 'PFIZER\\BIONTECH'
group by vax_manu, symptom
having reports > 1000
order by reports desc
limit 50

vax_manu,symptom,reports
PFIZER\BIONTECH,Headache,39579
PFIZER\BIONTECH,Fatigue,32024
PFIZER\BIONTECH,Pyrexia,29202
PFIZER\BIONTECH,Chills,26124
PFIZER\BIONTECH,Dizziness,26074
PFIZER\BIONTECH,Pain,25380
PFIZER\BIONTECH,Nausea,23620
PFIZER\BIONTECH,Pain in extremity,17822
PFIZER\BIONTECH,Myalgia,13222
PFIZER\BIONTECH,Arthralgia,13133


In [0]:
%sql 
select vax_manu, symptom, count(*) as reports 
from V_ALL_SYMPTOMS_STACK
where vax_manu = 'JANSSEN'
group by vax_manu, symptom
having reports > 1000
order by reports desc
limit 50

vax_manu,symptom,reports
JANSSEN,Headache,12493
JANSSEN,Pyrexia,10408
JANSSEN,Chills,9419
JANSSEN,Pain,8333
JANSSEN,Fatigue,8300
JANSSEN,Dizziness,6491
JANSSEN,Nausea,6469
JANSSEN,Pain in extremity,4569
JANSSEN,Myalgia,3282
JANSSEN,Injection site pain,2808


## Analysis 3 - Number of deaths

In [0]:
%sql --Strange cases in which the same report has different vaccines, probably due to faulty data.
select vaers_id, count(*) as cases
from V_VAERS_FULL_DATA
where died = 'Y'
group by vaers_id
having cases > 1
order by cases desc

vaers_id,cases
1358087,3
1207198,2
1433471,2
1345770,2


In [0]:
%sql 
select * 
from vaers_vax 
where vaers_id in (1358087, 1207198, 1433471, 1345770) 
order by vaers_id

VAERS_ID,VAX_MANU,VAX_NAME
1207198,PFIZER\BIONTECH,COVID19 (COVID19 (PFIZER-BIONTECH))
1207198,MODERNA,COVID19 (COVID19 (MODERNA))
1345770,UNKNOWN MANUFACTURER,COVID19 (COVID19 (UNKNOWN))
1345770,JANSSEN,COVID19 (COVID19 (JANSSEN))
1358087,JANSSEN,COVID19 (COVID19 (JANSSEN))
1358087,PFIZER\BIONTECH,COVID19 (COVID19 (PFIZER-BIONTECH))
1358087,MODERNA,COVID19 (COVID19 (MODERNA))
1433471,MODERNA,COVID19 (COVID19 (MODERNA))
1433471,PFIZER\BIONTECH,COVID19 (COVID19 (PFIZER-BIONTECH))


In [0]:
%sql select vax_manu, count(*) as cases
from V_VAERS_FULL_DATA
where died = 'Y'
group by vax_manu
order by cases desc

vax_manu,cases
PFIZER\BIONTECH,5814
MODERNA,2687
JANSSEN,523
UNKNOWN MANUFACTURER,29


In [0]:
%sql 
select substring(age_range, 3) as age_range, count(*) as cases 
from (
  select case
     when age_yrs < 18 then '1. Under 18'
     when age_yrs between 18 and 29 then '2. 18-29'
     when age_yrs between 30 and 49 then '3. 30-49'
     when age_yrs between 50 and 59 then '4. 50-59'
     when age_yrs between 60 and 69 then '5. 60-69'
     when age_yrs between 70 and 79 then '6. 70-79'
     when age_yrs between 80 and 89 then '7. 80-89'
     when age_yrs > 90 then '8. 90 and over'
     else '9. Undetermined'
     end as age_range
  from V_VAERS_FULL_DATA
  where died = 'Y'
  )
group by age_range
order by age_range

age_range,cases
Under 18,17
18-29,71
30-49,350
50-59,453
60-69,1014
70-79,1421
80-89,1688
90 and over,815
Undetermined,3224
